In [40]:
# Run this to cleanup environment if you need to rerun the code!
import os
os.system("rm *.mp4")
os.system("rm *.txt")
os.system("rm *.csv")
os.system("rm -rf result*")
if os.path.isdir("../runs"):
    os.system("rm -rf ../runs")

# Drone follow me using Kalman Filters

Multi-Object Tracking (MOT) is a core visual ability that humans poses to perform kinetic tasks and coordinate other tasks. The AI community has recognized the importance of MOT via a series of [competitions](https://motchallenge.net). 

In this assignment, the object class is `bicycle` and `car` the ability to track these objects  will be demonstrated using [Kalman Filters](https://en.wikipedia.org/wiki/Kalman_filter).  


## Task 1: Setup your development environment and store the test video locally (10 points)

Your environment must be docker based and you can use any TF2 or PT2 based docker container compatible with your environment. You can also use colab. 

In [ ]:
from pytube import YouTube
YouTube('https://www.youtube.com/watch?v=WeF4wpw7w9kWeF4wpw7w9k').streams.first().download()
YouTube('https://www.youtube.com/watch?v=2NFwY15tRtA').streams.first().download()
YouTube('https://www.youtube.com/watch?v=5dRramZVu2Q').streams.first().download()

In [42]:
input_file1 = 'Cyclist and vehicle Tracking - 1.mp4'
input_file2 = 'Cyclist and vehicle tracking - 2.mp4'
input_file3 = 'Drone Tracking Video.mp4'

## Task 2: Object Detection (40 points)

Perform object detection on the following videos. 

```{eval-rst}
.. youtube:: https://www.youtube.com/watch?v=WeF4wpw7w9kWeF4wpw7w9k
```

```{eval-rst}
.. youtube:: https://www.youtube.com/watch?v=2NFwY15tRtA
```

```{eval-rst}
.. youtube:: https://www.youtube.com/watch?v=5dRramZVu2Q
```
Split the videos into frames and use an object detector of your choice, in a framework of your choice to detect the cyclists.  

In [ ]:
from ultralytics import YOLO
import cv2

if os.path.isdir("./result1") != True:
    os.mkdir("./result1")

if os.path.isdir("./result2") != True:
    os.mkdir("./result2")

if os.path.isdir("./result3") != True:
    os.mkdir("./result3")

# Load the model
# mac = yolov8n.pt
# windows = yolov8x.pt
model = YOLO('yolov8x.pt')

cap = cv2.VideoCapture(input_file1)
n_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    
for frame in range(n_frames):
    ret, img = cap.read()
    if not ret:
        break

        #results = model(frame, device="mps")
        # use mps for macbook and device="0" for windows
    results = model.predict(source=img, save=True, device="0", conf=0.4)

    for result in results:
        boxes = result.boxes  # Boxes object for bounding box outputs
        masks = result.masks  # Masks object for segmentation masks outputs
        keypoints = result.keypoints  # Keypoints object for pose outputs
        probs = result.probs  # Probs object for classification outputs
        result.save(filename=f'./result1/result{frame}.jpg')  # save to disk
        result.save_txt('result1.txt', True)

cap.release()

cap = cv2.VideoCapture(input_file2)
n_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    
for frame in range(n_frames):
    ret, img = cap.read()
    if not ret:
        break

        #results = model(frame, device="mps")
        # use mps for macbook and device="0" for windows
    results = model.predict(source=img, save=True, device="0", conf=0.4)

    for result in results:
        boxes = result.boxes  # Boxes object for bounding box outputs
        masks = result.masks  # Masks object for segmentation masks outputs
        keypoints = result.keypoints  # Keypoints object for pose outputs
        probs = result.probs  # Probs object for classification outputs
        result.save(filename=f'./result2/result{frame}.jpg')  # save to disk
        result.save_txt('result2.txt', True)

cap.release()

cap = cv2.VideoCapture(input_file3)
n_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    
for frame in range(n_frames):
    ret, img = cap.read()
    if not ret:
        break

        #results = model(frame, device="mps")
        # use mps for macbook and device="0" for windows
    results = model.predict(source=img, save=True, device="0", conf=0.4)

    for result in results:
        boxes = result.boxes  # Boxes object for bounding box outputs
        masks = result.masks  # Masks object for segmentation masks outputs
        keypoints = result.keypoints  # Keypoints object for pose outputs
        probs = result.probs  # Probs object for classification outputs
        result.save(filename=f'./result3/result{frame}.jpg')  # save to disk
        result.save_txt('result3.txt', True)

cap.release()

In [ ]:
from ultralytics import YOLO

# Load a pretrained YOLOv8n model
model = YOLO('yolov8x.pt')

# Define source as YouTube video URL
source1 = input_file1

# Run inference on the source
results1 = model.predict(source1, save=True, device="0", conf=0.4)

# Define source as YouTube video URL
source2 = input_file2

# Run inference on the source
results2 = model.predict(source2, save=True, device="0", conf=0.4)

# Define source as YouTube video URL
source3 = input_file3

# Run inference on the source
results1 = model.predict(source3, save=True, device="0", conf=0.4)

In [ ]:
import subprocess
result_file1 = '../runs/detect/predict2/Cyclist and vehicle Tracking - 1.avi'
result_file2 = '../runs/detect/predict2/Cyclist and vehicle tracking - 2.avi'
result_file3 = '../runs/detect/predict2/Drone Tracking Video.avi'

subprocess.run(['ffmpeg', '-i', result_file1, 
                '-qscale', '0',
                '-loglevel', 'quiet',
                'output1.mp4'])
subprocess.run(['ffmpeg', '-i', result_file2, 
                '-qscale', '0',
                '-loglevel', 'quiet',
                'output2.mp4'])
subprocess.run(['ffmpeg', '-i', result_file3, 
                '-qscale', '0',
                '-loglevel', 'quiet',
                'output3.mp4'])

## Task 3: Kalman Filter (50 points)

Use the  [filterpy](https://filterpy.readthedocs.io/en/latest/kalman/KalmanFilter.html) library to implement Kalman filters that will track the cyclist and the vehicle (if present) in the video. You will need to use the detections from the previous task to initialize and run the Kalman filter. 

You need to deliver a video that contains the trajectory of the objects as a line that connects the pixels that the tracker indicated. You can use the `ffmpeg` command line tool and OpenCV to superpose the bounding box of the drone on the video as well as plot its trajectory. 

Suggest methods that you can use to address  false positives and how the tracker can help you in this regard.

You will need to have one Kalman filter to track each of the required and present objects (cyclist and vehicle).

## Extra Bonus (20 points)

```{eval-rst}
.. youtube:: https://www.youtube.com/watch?v=2hQx48U1L-Y2hQx48U1L-Y
```

The cyclist in the video goes in and out of occlusions. In addition the object is small making detections fairly problematic without finetuning and other optimizations.  Fintetuning involves using the pretrained model and training it further using images of cyclists from a training dataset such as [VisDrone](https://github.com/VisDrone/VisDrone-Dataset). At the same time,  reducing the number of classes to a much smaller number such as person & bicycle may help.  Also some 2 stage detectors may need to be further optimized in terms of parameters for small objects. See [this paper](https://www.mdpi.com/1424-8220/23/15/6887) for ideas around small object tracking. 

```{note}
The extra points can only be awarded in the category of `assignments` and cannot be used to compensate for any other category such as `exams`. 
```